In [20]:
import pandas as pd
df = pd.read_parquet("../data/dataset_tg.parquet")

In [21]:
df["id_channel"].value_counts()

id_channel
3     15747
1      8552
6      6105
4      5073
2      3916
18     2891
5      2476
Name: count, dtype: int64

In [22]:
df.head()

,Unnamed: 0.1,Unnamed: 0,message_id,views_o0,views_o1,views_o2,views_o3,forwards_o0,forwards_o1,forwards_o2,...,best_ctr_reactions_0_3,viral_ml,viral_final,llm_json,is_economic,topic,confidence,reason,economic_signals,noise_signals
0,0,0,00027151-a524-4d93-a820-116398fb81bb,492127.0,603269.0,619397.0,625475.0,381.0,415.0,449.0,...,NaN,-0.173209,0.406649,"{'confidence': 0.96, 'economic_signals': array...",False,None,0.96,Текст содержит лишь политические и военные выс...,NaN,NaN
1,1,2,0003b835-cf4a-43ff-b155-a144cf56b7f8,486195.0,507393.0,514469.0,517508.0,273.0,273.0,273.0,...,NaN,-0.150123,0.445509,"{'confidence': 0.9, 'economic_signals': array(...",False,None,0.90,Новость посвящена уголовному делу о хищении ср...,NaN,NaN
2,2,4,0005535a-74a9-4cb9-853f-ce04612d2f94,455967.0,609468.0,627631.0,634731.0,448.0,551.0,551.0,...,NaN,-0.170353,0.396754,"{'confidence': 0.95, 'economic_signals': array...",False,None,0.95,Новость описывает процесс поступления детей в ...,NaN,NaN
3,3,7,0007e2f8-787d-404f-91ff-e2582096a4a7,22376.0,23594.0,23735.0,23825.0,5.0,9.0,9.0,...,NaN,0.005396,0.703622,"{'confidence': 0.96, 'economic_signals': array...",True,Санкции и геополитика,0.96,Новость посвящена позиции Сербии по санкциям Е...,NaN,NaN
4,4,8,000884a5-8291-4ec1-805f-ac131112aaf7,19493.0,20605.0,NaN,NaN,64.0,67.0,NaN,...,0.00041,0.024601,0.680369,"{'confidence': 0.96, 'economic_signals': array...",True,Рынки капитала,0.96,Новость описывает падение фондового рынка и ра...,NaN,NaN


In [23]:
df.shape

(44760, 95)

In [24]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/cleaned_news_exp.csv")
texts = df["message"].fillna("").astype(str)

df_len = pd.DataFrame({
    "chars": texts.str.len(),
    "words": texts.str.split().map(len),
})

def qstats(s):
    return {
        "count": int(s.count()),
        "mean": float(s.mean()),
        "p50": float(s.quantile(0.50)),
        "p75": float(s.quantile(0.75)),
        "p90": float(s.quantile(0.90)),
        "p95": float(s.quantile(0.95)),
        "p99": float(s.quantile(0.99)),
        "max": float(s.max()),
    }

print("CHARS:", qstats(df_len["chars"]))
print("WORDS:", qstats(df_len["words"]))


CHARS: {'count': 18618, 'mean': 451.36663443978944, 'p50': 334.0, 'p75': 558.0, 'p90': 908.0, 'p95': 1127.1499999999978, 'p99': 2348.129999999979, 'max': 4067.0}
WORDS: {'count': 18618, 'mean': 63.3464926415297, 'p50': 47.0, 'p75': 77.0, 'p90': 124.29999999999927, 'p95': 158.0, 'p99': 317.8299999999981, 'max': 634.0}


In [25]:
! pip install torch accelerate transformers sentence_transformers

Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


### intfloat/multilingual-e5-small 
python embed_st.py   --input ../data/cleaned_news_exp.csv   --model intfloat/multilingual-e5-small   --prefix "passage: "   --out embeddings/emb_e5_small_fp16.npy --batch_size 16   --max_len 512

### intfloat/multilingual-e5-base
python embed_st.py   --input ../data/cleaned_news_exp.csv   --model intfloat/multilingual-e5-base   --prefix "passage: "   --out embeddings/emb_e5_base_fp16.npy --batch_size 16   --max_len 512

### intfloat/multilingual-e5-large
python embed_st.py   --input ../data/cleaned_news_exp.csv   --model intfloat/multilingual-e5-large   --prefix "passage: "   --out embeddings/emb_e5_large_fp16.npy  --batch_size 16   --max_len 512

### sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 
python embed_st.py   --input ../data/cleaned_news_exp.csv   --model sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2   --out embeddings/emb_minilm_fp16.npy  --batch_size 32   --max_len 512

### Alibaba-NLP/gte-multilingual-base
python embed_gte.py   --input ../data/cleaned_news_exp.csv   --out embeddings/emb_gte_fp16.npy   --rowmap embeddings/rowmap_gte.csv   --batch_size 4 --max_len 256

In [26]:
df["message_id"].nunique()

18618

In [27]:
df = df[["message_id", "id_channel", "message", "date", "topic"]]
df.head()

,message_id,id_channel,message,date,topic
0,0007e2f8-787d-404f-91ff-e2582096a4a7,18,Сербия согласна поддержать санкции Евросоюза п...,2025-07-26 07:01:09,Санкции и геополитика
1,000884a5-8291-4ec1-805f-ac131112aaf7,6,Китайский рынок акций упал сильнее всего с апр...,2025-09-04 10:16:56,Рынки капитала
2,000b0331-92a9-4eb4-9f58-d00811257758,18,Министерство труда США отменило рекомендации 2...,2025-05-29 04:05:09,Государственная экономическая политика
3,000b8df7-d902-41eb-b668-900614902f0a,6,Чистая прибыль Московской биржи по МСФО во вто...,2025-08-26 11:40:55,Корпоративные финансы
4,0011adea-7a98-4dcc-b753-905597b42788,4,"США хотят получить нефть и «всё, что угодно» о...",2025-02-22 20:57:18,Сырьевые рынки


In [28]:
rowmap = pd.read_csv("embeddings/rowmap.csv")
rowmap.head()

,row_id,message_id,date
0,0,0007e2f8-787d-404f-91ff-e2582096a4a7,2025-07-26 07:01:09
1,1,000884a5-8291-4ec1-805f-ac131112aaf7,2025-09-04 10:16:56
2,2,000b0331-92a9-4eb4-9f58-d00811257758,2025-05-29 04:05:09
3,3,000b8df7-d902-41eb-b668-900614902f0a,2025-08-26 11:40:55
4,4,0011adea-7a98-4dcc-b753-905597b42788,2025-02-22 20:57:18


In [29]:
# проверка согласованности
import numpy as np
import pandas as pd
from pathlib import Path

rowmap = pd.read_csv("embeddings/rowmap.csv")
N = len(rowmap)

emb_dir = Path("embeddings")
for p in sorted(emb_dir.glob("emb_*_fp16.npy")):
    E = np.load(p, mmap_mode="r")
    print(p.name, E.shape, "OK" if E.shape[0] == N else "MISMATCH")

emb_e5_base_fp16.npy (18618, 768) OK
emb_e5_large_fp16.npy (18618, 1024) OK
emb_e5_small_fp16.npy (18618, 384) OK
emb_gte_fp16.npy (18618, 768) OK
emb_minilm_fp16.npy (18618, 384) OK


In [11]:
import json, re, torch
from transformers import AutoTokenizer, AutoModelForCausalLM

JUDGE_MODEL = "Qwen/Qwen2.5-7B-Instruct"

judge_tokenizer = AutoTokenizer.from_pretrained(JUDGE_MODEL, trust_remote_code=True)
judge_model = AutoModelForCausalLM.from_pretrained(
    JUDGE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
    trust_remote_code=True,
)
judge_model.eval()

def _extract_first_json_obj(text: str):
    text = (text or "").strip()
    if text.startswith("{") and text.endswith("}"):
        try:
            return json.loads(text)
        except Exception:
            pass
    for m in re.finditer(r"\{.*?\}", text, flags=re.DOTALL):
        cand = m.group(0)
        try:
            return json.loads(cand)
        except Exception:
            continue
    return None

@torch.inference_mode()
def judge_pair(query: str, snippet: str, max_new_tokens: int = 40) -> int:
    user = f"Запрос:\n{query}\n\nДокумент:\n{snippet}\n"
    messages = [
        {"role": "system", "content": JUDGE_SYSTEM},
        {"role": "user", "content": user},
    ]
    prompt = judge_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = judge_tokenizer(prompt, return_tensors="pt").to(judge_model.device)

    out = judge_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        eos_token_id=judge_tokenizer.eos_token_id,
        pad_token_id=judge_tokenizer.eos_token_id,
    )
    gen = judge_tokenizer.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    data = _extract_first_json_obj(gen)

    if isinstance(data, dict) and data.get("relevance") in (0, 1, 2):
        return int(data["relevance"])

    return 0

def judge_with_llm(query: str, docs: list[str]) -> list[int]:
    return [judge_pair(query, d) for d in docs]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [12]:
JUDGE_SYSTEM = """Ты - строгий эксперт по информационному поиску по новостям об экономике.

Оцени, насколько найденный документ релевантен запросу.

Запрос:
{query}

Документ:
{snippet}

Оценка:
2 — документ напрямую отвечает на запрос или описывает то же событие.
1 — документ тематически близок, но не отвечает напрямую запросу.
0 — документ нерелевантен.

Верни ТОЛЬКО JSON и ничего более:
{"relevance": 0|1|2}
"""

In [37]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import re

DATA_PATH = Path("../data/cleaned_news_exp.csv")    
ROWMAP_PATH = Path("embeddings/rowmap.csv")
EMB_DIR = Path("embeddings")

K = 10
SNIPPET_CHARS = 1000


QUERY_SPECS = [
    "курс рубля к доллару",
    "укрепление рубля причины",
    "ослабление рубля после решений ФРС",
    "курс евро к рублю прогноз",
    "валютный рынок интервенции",
    "ключевая ставка ЦБ решение",
    "повышение ключевой ставки последствия",
    "снижение ключевой ставки эффект на кредиты",
    "инфляция в России ускорилась",
    "инфляция замедлилась причины",
    "индекс потребительских цен ИПЦ",
    "нефть Brent цена рост",
    "нефть Brent падение причины",
    "ОПЕК решение по добыче",
    "газ цены в Европе",
    "фондовый рынок России падение",
    "Мосбиржа индекс IMOEX рост",
    "акции банков динамика",
    "санкции влияние на экономику",
    "бюджет дефицит доходы расходы",
]


df = pd.read_csv(DATA_PATH, usecols=["message_id", "message"])
df["message"] = df["message"].fillna("").astype(str)

rowmap = pd.read_csv(ROWMAP_PATH, usecols=["row_id", "message_id", "date"])
rowmap["date"] = pd.to_datetime(rowmap["date"], errors="coerce")

m = rowmap.merge(df, on="message_id", how="left")
assert m["message"].notna().all(), "Some messages missing after merge"
texts = m["message"].tolist()

emb_files = sorted(EMB_DIR.glob("emb_*.npy"))
print("Found embeddings:", [p.name for p in emb_files])

from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModel

MODEL_REGISTRY = {
    "emb_e5_small_fp16.npy": ("st",  "intfloat/multilingual-e5-small", "query: "),
    "emb_e5_base_fp16.npy":  ("st",  "intfloat/multilingual-e5-base",  "query: "),
    "emb_e5_large_fp16.npy": ("st",  "intfloat/multilingual-e5-large", "query: "),
    "emb_minilm_fp16.npy":   ("st",  "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", ""),
    "emb_gte_fp16.npy":      ("gte", "Alibaba-NLP/gte-multilingual-base", ""),
}

_st_cache = {}
_gte_cache = None

def encode_query_st(model_name, prefix, query_text):
    if model_name not in _st_cache:
        mod = SentenceTransformer(model_name)
        _st_cache[model_name] = mod
    mod = _st_cache[model_name]
    q = (prefix + query_text) if prefix else query_text
    v = mod.encode([q], normalize_embeddings=True, show_progress_bar=False)
    return v.astype(np.float32)[0]

def mean_pool(last_hidden, attention_mask):
    mask = attention_mask.unsqueeze(-1).to(last_hidden.dtype)
    summed = (last_hidden * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-6)
    return summed / counts

@torch.inference_mode()
def encode_query_gte(model_name, query_text, max_len=256):
    global _gte_cache
    if _gte_cache is None:
        tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        mdl = AutoModel.from_pretrained(
            model_name, trust_remote_code=True, dtype=torch.float16, low_cpu_mem_usage=True
        )
        mdl.eval()
        _gte_cache = (tok, mdl)

    tok, mdl = _gte_cache
    inputs = tok([query_text], padding=True, truncation=True, max_length=max_len, return_tensors="pt")
    out = mdl(**inputs)
    emb = mean_pool(out.last_hidden_state, inputs["attention_mask"])
    emb = torch.nn.functional.normalize(emb, p=2, dim=1)
    return emb.cpu().numpy().astype(np.float32)[0]

def encode_query_for_embfile(embfile_name, query_text):
    typ, model_name, prefix = MODEL_REGISTRY[embfile_name]
    if typ == "st":
        return encode_query_st(model_name, prefix, query_text)
    elif typ == "gte":
        return encode_query_gte(model_name, query_text)
    else:
        raise ValueError("Unknown type")


def load_and_normalize_emb(path: Path):
    E = np.load(path, mmap_mode="r")
    X = E.astype(np.float32)
    X /= (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)
    return X

def topk_cosine(qvec, X, k=10):
    sims = X @ qvec
    idx = np.argpartition(-sims, kth=np.arange(k))[:k]
    idx = idx[np.argsort(-sims[idx])]
    return idx, sims[idx]

def snippet(t, n=SNIPPET_CHARS):
    t = (t or "").strip()
    t = re.sub(r"\s+", " ", t)
    return t[:n]



def dcg(rels):
    rels = np.array(rels, dtype=float)
    denom = np.log2(np.arange(2, len(rels) + 2))
    return float(np.sum((2**rels - 1) / denom))

def ndcg_at_k(rels, k=10):
    rels = rels[:k]
    ideal = sorted(rels, reverse=True)
    denom = dcg(ideal)
    return 0.0 if denom == 0 else dcg(rels) / denom

def precision_at_k(rels, k=10, thr=1):
    rels = np.array(rels[:k])
    return float(np.mean(rels >= thr))

def mrr_at_k(rels, k=10, thr=2):
    rels = rels[:k]
    for i, r in enumerate(rels, start=1):
        if r >= thr:
            return 1.0 / i
    return 0.0




results = []

for emb_path in tqdm(emb_files, desc="Embedders"):
    emb_name = emb_path.name
    if emb_name not in MODEL_REGISTRY:
        print("Skip (no registry entry):", emb_name)
        continue

    X = load_and_normalize_emb(emb_path)
    assert X.shape[0] == len(texts), f"N mismatch: {emb_name} {X.shape[0]} vs {len(texts)}"

    for q in tqdm(QUERY_SPECS, desc=f"Queries for {emb_name}", leave=False):
        qv = encode_query_for_embfile(emb_name, q)

        idx, sims = topk_cosine(qv, X, k=K)
        docs = [snippet(texts[i]) for i in idx]

        #LLM judge
        scores = judge_with_llm(q, docs) 
        assert isinstance(scores, list) and len(scores) == K
        assert all(s in (0,1,2) for s in scores)

        results.append({
            "emb": emb_name,
            "query": q,
            "P@10(rel>=1)": precision_at_k(scores, k=K, thr=1),
            "P@10(rel=2)": precision_at_k(scores, k=K, thr=2),
            "nDCG@10": ndcg_at_k(scores, k=K),
            "MRR@10(rel=2)": mrr_at_k(scores, k=K, thr=2),
        })

res = pd.DataFrame(results)


Found embeddings: ['emb_e5_base_fp16.npy', 'emb_e5_large_fp16.npy', 'emb_e5_small_fp16.npy', 'emb_gte_fp16.npy', 'emb_minilm_fp16.npy']


Queries for emb_gte_fp16.npy:   0%|          | 0/20 [00:00<?, ?it/s]Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

Embedders: 100%|██████████| 5/5 [03:53<00:00, 46.67s/it]                        


In [38]:
summary = (res.groupby("emb")[["P@10(rel>=1)", "P@10(rel=2)", "nDCG@10", "MRR@10(rel=2)"]]
             .mean()
             .sort_values("nDCG@10", ascending=False))

display(summary)

,P@10(rel>=1),P@10(rel=2),nDCG@10,MRR@10(rel=2)
emb,,,,
emb_e5_small_fp16.npy,0.920,0.595,0.954822,0.866667
emb_e5_large_fp16.npy,0.955,0.665,0.930934,0.872500
emb_e5_base_fp16.npy,0.920,0.685,0.927700,0.854167
emb_gte_fp16.npy,0.840,0.425,0.882691,0.622917
emb_minilm_fp16.npy,0.820,0.440,0.863247,0.708333


### Протестим на поиск по новостям

In [41]:
N_QUERIES = 50
RANDOM_SEED = 42

JUDGE_SYSTEM = """Ты — строгий эксперт по информационному поиску по экономическим новостям.

Оцени релевантность кандидата к исходной новости: насколько это тот же сюжет/инфоповод или его прямое развитие.

Исходная новость:
{query}

Кандидатная новость:
{snippet}

Оценка:
2 — тот же инфоповод/событие/заявление (включая перепечатки) ИЛИ прямое развитие/реакция/последствия именно этого инфоповода.
1 — общая тема совпадает (например ставка/рубль/нефть), но это другой инфоповод/другое событие.
0 — тема не совпадает или связь не прослеживается.

Если по тексту нельзя уверенно установить связь — выбирай более низкую оценку.

Верни ТОЛЬКО JSON:
{"relevance": 0|1|2}
"""

In [44]:
rng = np.random.default_rng(RANDOM_SEED)

valid_idx = np.array([i for i, t in enumerate(texts) if t and t.strip()])

query_indices = rng.choice(valid_idx, size=N_QUERIES, replace=False)

QUERY_NEWS = []
for qi in query_indices:
    QUERY_NEWS.append({
        "q_idx": int(qi), 
        "q_message_id": str(m.loc[qi, "message_id"]),
        "q_date": m.loc[qi, "date"],
        "q_text": texts[qi], 
    })

print("Sampled random news queries:", len(QUERY_NEWS))

Sampled random news queries: 50


In [57]:
def topk_cosine_excluding(qvec, X, k=10, exclude_idx=None):
    sims = X @ qvec
    if exclude_idx is not None:
        sims[int(exclude_idx)] = -1e9
    idx = np.argpartition(-sims, kth=np.arange(k))[:k]
    idx = idx[np.argsort(-sims[idx])]
    return idx, sims[idx]

results = []

for emb_path in tqdm(emb_files, desc="Embedders", leave=True):
    emb_name = emb_path.name
    X = load_and_normalize_emb(emb_path)

    for qobj in tqdm(QUERY_NEWS, desc=f"Random-news queries for {emb_name}", leave=False):
        qtext = qobj["q_text"]
        qidx = qobj["q_idx"]

        qv = X[qidx]  # news-as-query, НЕ кодируем моделью

        idx, sims = topk_cosine_excluding(qv, X, k=K, exclude_idx=qidx)
        docs = [snippet(texts[i]) for i in idx]

        scores = judge_with_llm(qtext, docs)
        assert isinstance(scores, list) and len(scores) == K
        assert all(s in (0,1,2) for s in scores)

        results.append({
            "emb": emb_name,
            "q_message_id": qobj["q_message_id"],
            "q_date": qobj["q_date"],
            "P@10(rel>=1)": precision_at_k(scores, k=K, thr=1),
            "P@10(rel=2)": precision_at_k(scores, k=K, thr=2),
            "nDCG@10": ndcg_at_k(scores, k=K),
            "MRR@10(rel=2)": mrr_at_k(scores, k=K, thr=2),
        })


Embedders: 100%|██████████| 5/5 [09:14<00:00, 110.84s/it]                                   


In [58]:
res = pd.DataFrame(results)

summary = (res.groupby("emb")[["P@10(rel>=1)", "P@10(rel=2)", "nDCG@10", "MRR@10(rel=2)"]]
             .mean()
             .sort_values("nDCG@10", ascending=False))

display(summary)

,P@10(rel>=1),P@10(rel=2),nDCG@10,MRR@10(rel=2)
emb,,,,
emb_e5_large_fp16.npy,0.956,0.226,0.952416,0.554167
emb_e5_base_fp16.npy,0.962,0.200,0.948884,0.529667
emb_e5_small_fp16.npy,0.974,0.190,0.940694,0.457667
emb_minilm_fp16.npy,0.938,0.162,0.936521,0.426714
emb_gte_fp16.npy,0.866,0.140,0.921675,0.416000


### Протестим на топ-50 две топовых модели e5 (base vs large)

In [60]:
@torch.inference_mode()
def judge_pairs_batched(pairs, batch_size=32, max_new_tokens=40):
    out_scores = []
    for i in range(0, len(pairs), batch_size):
        chunk = pairs[i:i+batch_size]

        prompts = []
        for q, sn in chunk:
            user = f"Запрос:\n{q}\n\nДокумент:\n{sn}\n"
            messages = [
                {"role": "system", "content": JUDGE_SYSTEM},
                {"role": "user", "content": user},
            ]
            prompts.append(
                judge_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            )

        enc = judge_tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
        ).to(judge_model.device)

        gen_ids = judge_model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            eos_token_id=judge_tokenizer.eos_token_id,
            pad_token_id=judge_tokenizer.eos_token_id,
        )

        for b in range(len(chunk)):
            prompt_len = int(enc["attention_mask"][b].sum().item())
            gen_txt = judge_tokenizer.decode(gen_ids[b][prompt_len:], skip_special_tokens=True)
            data = _extract_first_json_obj(gen_txt)
            rel = int(data["relevance"]) if isinstance(data, dict) and data.get("relevance") in (0,1,2) else 0
            out_scores.append(rel)

    return out_scores

In [61]:
JUDGE_SYSTEM_TOPIC = """Ты — строгий эксперт по информационному поиску по экономическим новостям.

Оцени релевантность документа запросу-топику.

Запрос:
{query}

Документ:
{snippet}

Оценка:
2 — документ прямо про этот запрос (основная тема совпадает; содержит ключевой факт/событие по запросу).
1 — документ частично связан или близкий контекст, но не про запрос напрямую.
0 — нерелевантно.

Если сомневаешься — ставь более низкую оценку.
Верни ТОЛЬКО JSON:
{"relevance": 0|1|2}
"""

JUDGE_SYSTEM_NEWS = """Ты — строгий эксперт по информационному поиску по экономическим новостям.

Оцени, относится ли кандидатная новость к тому же инфоповоду (событию/решению/заявлению), что и исходная новость, либо является прямым развитием/реакцией на него.

Исходная новость:
{query}

Кандидатная новость:
{snippet}

Оценка:
2 — тот же инфоповод/перепечатка ИЛИ прямое развитие/реакция/последствия именно этого инфоповода.
1 — общая тема совпадает (например ставка/рубль/нефть), но это другой инфоповод.
0 — нерелевантно.

Если сомневаешься — ставь более низкую оценку.
Верни ТОЛЬКО JSON:
{"relevance": 0|1|2}
"""

In [62]:
N_QUERIES_TOTAL = 50
N_MANUAL = 20
N_AUTO = N_QUERIES_TOTAL - N_MANUAL
K = 50

SNIPPET_CHARS = 1000

def snippet(t, n=SNIPPET_CHARS):
    t = (t or "").strip()
    t = re.sub(r"\s+", " ", t)
    return t[:n]

def topk_cosine(qvec, X, k=50, exclude_idx=None):
    sims = X @ qvec
    if exclude_idx is not None:
        sims[int(exclude_idx)] = -1e9
    idx = np.argpartition(-sims, kth=k-1)[:k]
    idx = idx[np.argsort(-sims[idx])]
    return idx

rng = np.random.default_rng(SEED)

#топики
manual_queries = [{"type": "topic", "text": q, "source_idx": None} for q in QUERY_SPECS[:N_MANUAL]]

#случайные новости
valid_idx = np.array([i for i, t in enumerate(texts) if t and t.strip()])
auto_idx = rng.choice(valid_idx, size=N_AUTO, replace=False)

auto_queries = []
for i in auto_idx:
    qtxt = snippet(texts[i], 1500)
    auto_queries.append({"type": "news", "text": qtxt, "source_idx": int(i)})

queries = manual_queries + auto_queries
assert len(queries) == N_QUERIES_TOTAL

X_base  = load_and_normalize_emb(EMB_DIR / "emb_e5_base_fp16.npy")
X_large = load_and_normalize_emb(EMB_DIR / "emb_e5_large_fp16.npy")
assert X_base.shape[0] == len(texts)
assert X_large.shape[0] == len(texts)

enc_base  = SentenceTransformer("intfloat/multilingual-e5-base")
enc_large = SentenceTransformer("intfloat/multilingual-e5-large")

def encode_e5_query(encoder, q: str) -> np.ndarray:
    v = encoder.encode(["query: " + q], normalize_embeddings=True, show_progress_bar=False)
    return v.astype(np.float32)[0]

rows = []

for qobj in tqdm(queries, desc="Queries"):
    q_text = qobj["text"]
    q_type = qobj["type"]
    exclude_idx = qobj["source_idx"]

    qv_b = encode_e5_query(enc_base,  q_text)
    qv_l = encode_e5_query(enc_large, q_text)

    idx_b = topk_cosine(qv_b, X_base,  k=K, exclude_idx=exclude_idx)
    idx_l = topk_cosine(qv_l, X_large, k=K, exclude_idx=exclude_idx)

    docs_b = [snippet(texts[i]) for i in idx_b]
    docs_l = [snippet(texts[i]) for i in idx_l]


    if q_type == "news":
        JUDGE_SYSTEM = JUDGE_SYSTEM_NEWS
        q_label = "Исходная новость"
    else:
        JUDGE_SYSTEM = JUDGE_SYSTEM_TOPIC
        q_label = "Запрос"


    pairs_b = [(f"{q_label}:\n{q_text}", f"{d}") for d in docs_b]
    pairs_l = [(f"{q_label}:\n{q_text}", f"{d}") for d in docs_l]

    scores_b = judge_pairs_batched(pairs_b, batch_size=32)
    scores_l = judge_pairs_batched(pairs_l, batch_size=32)

    rows.append({
        "q_type": q_type,
        "query": q_text,
        "ndcg_base": ndcg_at_k(scores_b, K),
        "ndcg_large": ndcg_at_k(scores_l, K),
        "mrr_base": mrr_at_k(scores_b, K, thr=2),
        "mrr_large": mrr_at_k(scores_l, K, thr=2),
        "p_base_2": precision_at_k(scores_b, K, thr=2),
        "p_large_2": precision_at_k(scores_l, K, thr=2),
        "p_base_1": precision_at_k(scores_b, K, thr=1),
        "p_large_1": precision_at_k(scores_l, K, thr=1),
    })


res = pd.DataFrame(rows)

#сводка
summary_all = res[["ndcg_base","ndcg_large","mrr_base","mrr_large","p_base_2","p_large_2","p_base_1","p_large_1"]].mean()
display(summary_all)

#сводка по типам запросов
summary_by_type = res.groupby("q_type")[["ndcg_base","ndcg_large","mrr_base","mrr_large","p_base_2","p_large_2","p_base_1","p_large_1"]].mean()
display(summary_by_type)


Queries: 100%|██████████| 50/50 [09:10<00:00, 11.01s/it]


ndcg_base     0.674136
ndcg_large    0.674274
mrr_base      0.508694
mrr_large     0.506842
p_base_2      0.200000
p_large_2     0.204000
p_base_1      0.426800
p_large_1     0.421200
dtype: float64

,ndcg_base,ndcg_large,mrr_base,mrr_large,p_base_2,p_large_2,p_base_1,p_large_1
q_type,,,,,,,,
news,0.548237,0.543735,0.290513,0.309181,0.048,0.049333,0.231333,0.208667
topic,0.862983,0.870083,0.835965,0.803333,0.428,0.436000,0.720000,0.740000


In [63]:
summary = pd.DataFrame({
    "metric": ["nDCG@50", "MRR@50(rel=2)", "P@50(rel=2)", "P@50(rel>=1)"],
    "e5_base": [
        res["ndcg_base"].mean(),
        res["mrr_base"].mean(),
        res["p_base_2"].mean(),
        res["p_base_1"].mean(),
    ],
    "e5_large": [
        res["ndcg_large"].mean(),
        res["mrr_large"].mean(),
        res["p_large_2"].mean(),
        res["p_large_1"].mean(),
    ],
})
display(summary)

print("Win-rate base > large | nDCG:", float((res["ndcg_base"] > res["ndcg_large"]).mean()))
print("Win-rate base > large | MRR :", float((res["mrr_base"]  > res["mrr_large"]).mean()))
print("Win-rate base > large | P@50(2):", float((res["p_base_2"] > res["p_large_2"]).mean()))

,metric,e5_base,e5_large
0,nDCG@50,0.674136,0.674274
1,MRR@50(rel=2),0.508694,0.506842
2,P@50(rel=2),0.200000,0.204000
3,P@50(rel>=1),0.426800,0.421200


Win-rate base > large | nDCG: 0.46
Win-rate base > large | MRR : 0.2
Win-rate base > large | P@50(2): 0.22


In [64]:
TOPN_POOL = 300  

def recall_at_k(scores, k, total_rel, thr=1):
    if total_rel <= 0:
        return 0.0
    s = np.array(scores[:k])
    return float(np.sum(s >= thr) / total_rel)

def f1(p, r):
    return 0.0 if (p + r) == 0 else float(2 * p * r / (p + r))

rows_pool = []

for qobj in tqdm(queries, desc="Pool-eval queries"):
    q_text = qobj["text"]
    q_type = qobj["type"]
    exclude_idx = qobj["source_idx"]

    if q_type == "news":
        JUDGE_SYSTEM = JUDGE_SYSTEM_NEWS
        q_label = "Исходная новость"
    else:
        JUDGE_SYSTEM = JUDGE_SYSTEM_TOPIC
        q_label = "Запрос"

    qv_b = encode_e5_query(enc_base,  q_text)
    qv_l = encode_e5_query(enc_large, q_text)

    idx_b_pool = topk_cosine(qv_b, X_base,  k=TOPN_POOL, exclude_idx=exclude_idx)
    idx_l_pool = topk_cosine(qv_l, X_large, k=TOPN_POOL, exclude_idx=exclude_idx)

    pool_ids = list(dict.fromkeys(list(idx_b_pool) + list(idx_l_pool)))
    pool_docs = [snippet(texts[i]) for i in pool_ids]

    pool_pairs = [(f"{q_label}:\n{q_text}", d) for d in pool_docs]
    pool_scores = judge_pairs_batched(pool_pairs, batch_size=32)
    pool_scores = list(map(int, pool_scores))

    score_map = {doc_id: pool_scores[j] for j, doc_id in enumerate(pool_ids)}

    total_rel1 = int(np.sum(np.array(pool_scores) >= 1))
    total_rel2 = int(np.sum(np.array(pool_scores) >= 2))

    idx_b = list(idx_b_pool[:K])
    idx_l = list(idx_l_pool[:K])

    scores_b = [score_map.get(doc_id, 0) for doc_id in idx_b]
    scores_l = [score_map.get(doc_id, 0) for doc_id in idx_l]

    p_b_1 = precision_at_k(scores_b, K, thr=1)
    p_l_1 = precision_at_k(scores_l, K, thr=1)
    p_b_2 = precision_at_k(scores_b, K, thr=2)
    p_l_2 = precision_at_k(scores_l, K, thr=2)

    r_b_1 = recall_at_k(scores_b, K, total_rel1, thr=1)
    r_l_1 = recall_at_k(scores_l, K, total_rel1, thr=1)
    r_b_2 = recall_at_k(scores_b, K, total_rel2, thr=2)
    r_l_2 = recall_at_k(scores_l, K, total_rel2, thr=2)

    f_b_1 = f1(p_b_1, r_b_1)
    f_l_1 = f1(p_l_1, r_l_1)
    f_b_2 = f1(p_b_2, r_b_2)
    f_l_2 = f1(p_l_2, r_l_2)

    ndcg_b = ndcg_at_k(scores_b, K)
    ndcg_l = ndcg_at_k(scores_l, K)
    mrr_b  = mrr_at_k(scores_b, K, thr=2)
    mrr_l  = mrr_at_k(scores_l, K, thr=2)

    overlap = len(set(idx_b).intersection(set(idx_l))) / float(K)

    rows_pool.append({
        "q_type": q_type,
        "query": q_text,
        "pool_size": len(pool_ids),
        "pool_rel1": total_rel1,
        "pool_rel2": total_rel2,
        "overlap@50": overlap,

        # old
        "ndcg_base": ndcg_b,
        "ndcg_large": ndcg_l,
        "mrr_base": mrr_b,
        "mrr_large": mrr_l,

        # precision
        "p_base_1": p_b_1,
        "p_large_1": p_l_1,
        "p_base_2": p_b_2,
        "p_large_2": p_l_2,

        # recall
        "r_base_1": r_b_1,
        "r_large_1": r_l_1,
        "r_base_2": r_b_2,
        "r_large_2": r_l_2,

        # f1
        "f1_base_1": f_b_1,
        "f1_large_1": f_l_1,
        "f1_base_2": f_b_2,
        "f1_large_2": f_l_2,
    })

res_pool = pd.DataFrame(rows_pool)

cols = [
    "ndcg_base","ndcg_large","mrr_base","mrr_large",
    "p_base_2","p_large_2","p_base_1","p_large_1",
    "r_base_2","r_large_2","r_base_1","r_large_1",
    "f1_base_2","f1_large_2","f1_base_1","f1_large_1",
    "overlap@50","pool_rel1","pool_rel2","pool_size"
]

summary_all = res_pool[cols].mean()
display(summary_all)

summary_by_type = res_pool.groupby("q_type")[cols].mean()
display(summary_by_type)

def win_rate(df, a, b):
    return float(np.mean(df[a] > df[b]))

print("Win-rate large > base | Recall@50(rel>=1):", win_rate(res_pool, "r_base_1", "r_large_1"))
print("Win-rate large > base | Recall@50(rel=2) :", win_rate(res_pool, "r_base_2", "r_large_2"))
print("Win-rate large > base | F1@50(rel>=1)    :", win_rate(res_pool, "f1_base_1", "f1_large_1"))
print("Win-rate large > base | F1@50(rel=2)     :", win_rate(res_pool, "f1_base_2", "f1_large_2"))


Pool-eval queries: 100%|██████████| 50/50 [40:06<00:00, 48.14s/it]


ndcg_base       0.672879
ndcg_large      0.667875
mrr_base        0.508694
mrr_large       0.523509
p_base_2        0.200400
p_large_2       0.199200
p_base_1        0.422000
p_large_1       0.407200
r_base_2        0.367043
r_large_2       0.349133
r_base_1        0.149344
r_large_1       0.127296
f1_base_2       0.176353
f1_large_2      0.173470
f1_base_1       0.194604
f1_large_1      0.177866
overlap@50      0.512400
pool_rel1     145.260000
pool_rel2      37.280000
pool_size     433.600000
dtype: float64

,ndcg_base,ndcg_large,mrr_base,mrr_large,p_base_2,p_large_2,p_base_1,p_large_1,r_base_2,r_large_2,r_base_1,r_large_1,f1_base_2,f1_large_2,f1_base_1,f1_large_1,overlap@50,pool_rel1,pool_rel2,pool_size
q_type,,,,,,,,,,,,,,,,,,,,
news,0.546315,0.534196,0.290513,0.336959,0.048,0.046667,0.225333,0.190667,0.370636,0.339182,0.145212,0.107025,0.069667,0.067683,0.155428,0.124934,0.494667,82.20,7.366667,444.833333
topic,0.862726,0.868392,0.835965,0.803333,0.429,0.428000,0.717000,0.732000,0.361654,0.364060,0.155543,0.157703,0.336383,0.332149,0.253367,0.257265,0.539000,239.85,82.150000,416.750000


Win-rate large > base | Recall@50(rel>=1): 0.56
Win-rate large > base | Recall@50(rel=2) : 0.22
Win-rate large > base | F1@50(rel>=1)    : 0.56
Win-rate large > base | F1@50(rel=2)     : 0.22


## Выбираем e5_large как эмбеддер для проекта так как будем концентрироваться на топиках, а там качество выше (допроверить)